In [230]:
from os import walk

fitsfiles = [] 
for (dirpath, dirnames, filenames) in walk('./fitsfiles'):
    for file in filenames:    
        if(file.endswith('.fits')):
            if('pfss_intoout' in file):
                fitsfiles.append((dirpath + '/' + file, 'PFSS_IO'))
            elif('pfss_outtoin' in file):
                fitsfiles.append((dirpath + '/' + file, 'PFSS_OI'))
            elif('scs_outtoin' in file):
                fitsfiles.append((dirpath + '/' + file, 'SCS_OI'))
    break

### Choose indices

In [246]:
def everynth(step):
    return (range(0,16200, step), 'step' + str(step))

indices = everynth(9)

### Defintion from OpenSpace function FieldlinesState::saveStateToOsfls
    File is structured like this: (for version 0)
    0. int                    - version number of binary state file! (in case something needs to be altered in the future, then increase CurrentVersion)
    1. double                 - _triggerTime
    2. int                    - _model
    3. bool                   - _isMorphable
    4. size_t                 - Number of lines in the state  == _lineStart.size() == _lineCount.size()
    5. size_t                 - Total number of vertex points  == _vertexPositions.size() == _extraQuantities.size()
    6. size_t                 - Number of extra quantites  == _extraQuantities.size() == _extraQuantityNames.size()
    7. site_t                 - Number of total bytes that ALL _extraQuantityNames  consists of (Each such name is stored as a c_str which means it ends with the null char '\0' )
    8. std::vector<GLint>     - _lineStart
    9. std::vector<GLsizei>   - _lineCount
    10. std::vector<glm::vec3> - _vertexPositions
    11. std::vector<float>     - _extraQuantities
    12. array of c_str         - Strings naming the extra quantities (elements of _extraQuantityNames). Each string ends with null char '\0'

In [232]:
import math, numpy

def sph2cart(coord):
    return [
         coord[2] * math.sin(coord[1]) * math.cos(coord[0]), 
         coord[2] * math.sin(coord[1]) * math.sin(coord[0]),
         coord[2] * math.cos(coord[1])
    ]

def coord2datarow(coord):
    sph = sph2cart(coord[0:3])
    sph.append(abs(coord[5]))
    return [numpy.float32(x) for x in sph] 

In [233]:
from enum import Enum
class Model(Enum):
    Batsrus = 0
    Enlil = 1
    Pfss = 2
    Wsa = 3
    Invalid = 5

In [248]:
from astropy.io import fits
from astropy.time import Time

def toOsfls(filename, modelname, indices,  typename):
    fl_fits = fits.open(filename)
    fl_data = fl_fits[0].data
    time = Time(fl_fits[0].header['OBSJUL'], format='jd')
    fl_fits.close()

    versionNumber = 0
    triggerTime = time.jd # 2458606.1667
    pathSafeTimeString = time.fits.replace(':', '-') #'2019-05-02T16-00-02.880'
    fileName = pathSafeTimeString + '.osfls'

    model = Model.Wsa.value
    isMorphable = False

    nVert = 0
    lineStart = []
    lineCount = []
    vertexPositions = []
    extraQuantities = []

    for i in indices:
        points = [coord2datarow(pt) for pt in fl_data[i] if pt[0] > -900] 
        if (len(points) < 2): continue
        lineStart.append(nVert)
        nVert += len(points)
        lineCount.append(len(points))
        [vertexPositions.extend(pt[0:3]) for pt in points] # extend to unfold elements
        [extraQuantities.append(pt[3]) for pt in points]

    nLines = len(lineStart)

    nExtras = 1
    extraQuantityNames = ['b-f mag\0']
    nStringBytes = sum([len(s) for s in extraQuantityNames])
    allNamesInOne = ''
    for s in extraQuantityNames:
        allNamesInOne += s
    
    # Prepare data for writing to binary. Using Struct and pack
    typestr = '= i d i ? Q Q Q Q %sl %sL %sf %sf %ss' % (nLines, nLines, 3*nVert, nVert, nStringBytes)
    struct_to_write = struct.Struct(typestr)
    print('Format string  :', struct_to_write.format)
    print('Uses           :', struct_to_write.size, 'bytes')
    values_to_write = (versionNumber, triggerTime, model, isMorphable, nLines, nVert, nExtras, nStringBytes)
    values_to_write += (*lineStart, *lineCount, *vertexPositions, *extraQuantities, allNamesInOne.encode('utf-8'))
    
    buffer = ctypes.create_string_buffer(struct_to_write.size)    
    struct_to_write.pack_into(b, 0, *values_to_write)
    
    fout = open('./' + modelname + '/' + typename + '_' + pathSafeTimeString +'.osfls', 'wb')
    fout.write(buffer)
    fout.close()


### Make into osfls format

In [249]:
import time
start_time = time.time()
for fitsfile in fitsfiles:
    toOsfls(fitsfile[0], fitsfile[1], indices[0], indices[1])
    print('Finished converting {} after {} seconds: '.format(fitsfile[0],time.time()-start_time))
print("Execution time for type {}: {} seconds".format(indices[1], time.time()-start_time))


Format string  : = i d i ? Q Q Q Q 1800l 1800L 860907f 286969f 8s
Uses           : 4605961 bytes
Finished converting ./fitsfiles/201905020800R000_trace_pfss_outtoin_tracing.fits after 1.6911678314208984 seconds: 
Format string  : = i d i ? Q Q Q Q 1800l 1800L 1135710f 378570f 8s
Uses           : 6071577 bytes
Finished converting ./fitsfiles/201905020800R000_trace_scs_outtoin_tracing.fits after 3.8425137996673584 seconds: 
Format string  : = i d i ? Q Q Q Q 1800l 1800L 1136169f 378723f 8s
Uses           : 6074025 bytes
Finished converting ./fitsfiles/201905021200R000_trace_scs_outtoin_tracing.fits after 6.456794023513794 seconds: 
Format string  : = i d i ? Q Q Q Q 1800l 1800L 860328f 286776f 8s
Uses           : 4602873 bytes
Finished converting ./fitsfiles/201905021600R000_trace_pfss_outtoin_tracing.fits after 8.116503953933716 seconds: 
Format string  : = i d i ? Q Q Q Q 1800l 1800L 860064f 286688f 8s
Uses           : 4601465 bytes
Finished converting ./fitsfiles/201905022000R000_trac